<a href="https://colab.research.google.com/github/gitHubAndyLee2020/LLM_Reward_Model_Research/blob/main/Mistral_7B_Instruct_v0_1_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset Initialization

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# The dataset identifier is usually in the format "user/dataset_name"
dataset_identifier = "Open-Orca/OpenOrca"
dataset = load_dataset(dataset_identifier)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import pandas as pd

# Convert the "train" split to a DataFrame
df_train = dataset["train"].to_pandas()

# Display the first few rows
df_train.head()

,id,system_prompt,question,response
0,niv.242684,,You will be given a definition of a task first...,"[\n [""AFC Ajax (amateurs)"", ""has ground"", ""Sp..."
1,flan.564327,You are an AI assistant. You will be given a t...,Generate an approximately fifteen-word sentenc...,Midsummer House is a moderately priced Chinese...
2,flan.1875913,"You are a helpful assistant, who always provid...",What happens next in this paragraph?\n\nShe th...,C. She then dips the needle in ink and using t...
3,t0.408370,You are an AI assistant. You will be given a t...,Please answer the following question: I want t...,"Based on the passage, discuss the primary moti..."
4,cot.86217,You are an AI assistant that helps people find...,James runs a TV show and there are 5 main char...,"James pays the minor characters $15,000 each e..."


### Model Initialization

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

(…)B-Instruct-v0.1/resolve/main/config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

(…)-v0.1/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

(…)nstruct-v0.1/resolve/main/tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

(…)0.1/resolve/main/special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
%%time

sys_prompt = "You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old."
prompt = "What kind of task would test someone's ability to perform physical reasoning?"

prefix = "<|im_start|>"
suffix = "<|im_end|>\n"
sys_format = prefix + "system\n" + sys_prompt + suffix
user_format = prefix + "user\n" + prompt + suffix
assistant_format = prefix + "assistant\n"
input_text = sys_format + user_format + assistant_format

messages = [
    {"role": "user", "content": input_text},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, use_cache=True, do_sample=True,
    temperature=0.2, top_p=0.95)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] <|im_start|>system
You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.<|im_end|>
<|im_start|>user
What kind of task would test someone's ability to perform physical reasoning?<|im_end|>
<|im_start|>assistant
 [/INST] Physical reasoning is about using your body and movements to solve problems. A task that would test someone's ability to perform physical reasoning could be something like trying to balance a stack of books on top of each other, or trying to solve a puzzle that involves moving pieces to fit together in a certain way. Another example could be trying to follow a set of instructions to assemble a toy or piece of furniture. These tasks require you to think about how your body moves and how different objects interact with each other to achieve a goal.</s>
CPU times: user 11.9 s, sys: 1.37 s, total: 13.2 s
Wall time: 14.6 s


### Generating Responses

In [ ]:
def generate_response(sys_prompt, prompt):
  prefix = "<|im_start|>"
  suffix = "<|im_end|>\n"
  sys_format = prefix + "system\n" + sys_prompt + suffix
  user_format = prefix + "user\n" + prompt + suffix
  assistant_format = prefix + "assistant\n"
  input_text = sys_format + user_format + assistant_format

  messages = [
      {"role": "user", "content": input_text},
  ]

  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  model_inputs = encodeds.to(device)
  model.to(device)

  generated_ids = model.generate(model_inputs, max_new_tokens=1000, use_cache=True, do_sample=True,
    temperature=0.2, top_p=0.95)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0]

In [ ]:
import re

def extract_response(text):
  result = re.search(r'\[/INST\](.*?)</s>', text, re.DOTALL)
  if result:
      extracted_text = result.group(1).strip()
      return extracted_text
  else:
      print("Pattern not found.")
      return ''

In [ ]:
def get_response(sys_prompt, prompt):
  response = generate_response(sys_prompt, prompt)
  return extract_response(response)

In [ ]:
import os

def generate_answers(df_train, START_IDX, END_IDX):
    # Check if data folder exists. If not, create it.
    if not os.path.exists("data"):
        os.makedirs("data")

    for idx in range(START_IDX, END_IDX):
        sys_prompt = df_train["system_prompt"][idx]
        prompt = df_train["question"][idx]
        answer = ""
        if (len(sys_prompt.split()) + len(prompt.split())) <= 2000: # Respects the context window size of the model
          answer = get_response(sys_prompt, prompt)
        else:
          print(f"{idx} has too large prompt; skipping...")
        with open(f"data/{idx}_mistral_7b_instruct_v0_1.txt", "w", encoding="utf-8") as f:
            f.write(answer)

In [ ]:
START_IDX = 0
END_IDX = 100

In [ ]:
%%time

generate_answers(df_train, START_IDX, END_IDX)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and 

977 has too large prompt; skipping...


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and 

Pattern not found.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and 

CPU times: user 2min 21s, sys: 911 ms, total: 2min 22s
Wall time: 2min 22s


In [ ]:
# If you get "NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968" error, run this to fix the problem

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!zip -r data.zip data/

  adding: data/ (stored 0%)
  adding: data/982_mistral_7b_instruct_v0_1.txt (stored 0%)
  adding: data/995_mistral_7b_instruct_v0_1.txt (deflated 11%)
  adding: data/975_mistral_7b_instruct_v0_1.txt (deflated 46%)
  adding: data/980_mistral_7b_instruct_v0_1.txt (deflated 54%)
  adding: data/993_mistral_7b_instruct_v0_1.txt (deflated 37%)
  adding: data/981_mistral_7b_instruct_v0_1.txt (deflated 44%)
  adding: data/976_mistral_7b_instruct_v0_1.txt (deflated 21%)
  adding: data/985_mistral_7b_instruct_v0_1.txt (deflated 40%)
  adding: data/990_mistral_7b_instruct_v0_1.txt (stored 0%)
  adding: data/978_mistral_7b_instruct_v0_1.txt (deflated 40%)
  adding: data/992_mistral_7b_instruct_v0_1.txt (stored 0%)
  adding: data/986_mistral_7b_instruct_v0_1.txt (deflated 57%)
  adding: data/997_mistral_7b_instruct_v0_1.txt (deflated 59%)
  adding: data/999_mistral_7b_instruct_v0_1.txt (deflated 9%)
  adding: data/987_mistral_7b_instruct_v0_1.txt (deflated 46%)
  adding: data/983_mistral_7b_instruc

### Debug